In [1]:
#导入包
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
import random
from selenium.webdriver.chrome.service import Service
import pandas as pd
import datetime
import zipfile
import string
import requests

In [24]:
def create_proxyauth_extension(proxy_host, proxy_port,
                               proxy_username, proxy_password,
                               scheme='http', plugin_path=None):
    # 该配置不用改，可以直接用
    if plugin_path is None:
        plugin_path = 'chrome_proxyauth_plugin.zip'

    manifest_json = """
    {
        "version": "1.0.0",
        "manifest_version": 2,
        "name": "test Proxy",
        "permissions": [
            "proxy",
            "tabs",
            "unlimitedStorage",
            "storage",
            "<all_urls>",
            "webRequest",
            "webRequestBlocking"
        ],
        "background": {
            "scripts": ["background.js"]
        },
        "minimum_chrome_version":"22.0.0"
    }
    """

    background_js = string.Template(
        """
        var config = {
                mode: "fixed_servers",
                rules: {
                  singleProxy: {
                    scheme: "${scheme}",
                    host: "${host}",
                    port: parseInt(${port})
                  },
                  bypassList: ["foobar.com"]
                }
              };
        chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});
        function callbackFn(details) {
            return {
                authCredentials: {
                    username: "${username}",
                    password: "${password}"
                }
            };
        }
        chrome.webRequest.onAuthRequired.addListener(
                    callbackFn,
                    {urls: ["<all_urls>"]},
                    ['blocking']
        );
        """
    ).substitute(
        host=proxy_host,
        port=proxy_port,
        username=proxy_username,
        password=proxy_password,
        scheme=scheme,
    )
    with zipfile.ZipFile(plugin_path, 'w') as zp:
        zp.writestr("manifest.json", manifest_json)
        zp.writestr("background.js", background_js)

    return plugin_path


In [25]:
proxyauth_plugin_path = create_proxyauth_extension(
        proxy_host="geo.iproyal.com",
        proxy_port=12321,
        proxy_username="wangzx",        #代理账号
        proxy_password="asdf1234_country-hk_city-hongkong"       #代理密码

        # proxy_host="proxy.ipipgo.com",
        # proxy_port=31212,
        # proxy_username="customer-d47eec-country-JP",
        # proxy_password="33577fac"
    )

In [48]:
def get_browser(url:str='https://www.apple.com/hk/shop/buy-iphone/iphone-14-pro'):
    #随机获取用户表示ua
    UserAgents=pd.read_csv('/Users/zuimingze/python练练/tools/ua_string.csv')
    ua=UserAgents.iloc[random.randint(0,46478)][0]

    ch_options = Options()
    #去除受到自动软件控制
    ch_options.add_experimental_option('excludeSwitches', ['enable-automation'])
    ch_options.add_argument('user-agent={}'.format(ua))
    ch_options.add_argument('--incognito')
    #ch_options.page_load_strategy = 'eager'
    #忽略安全提示
    ch_options.add_argument("--ignore-certificate-errors") 
    
    #####添加代理插件
    
   
   
    #ch_options.add_extension('chrome_proxyauth_plugin.zip')
    driver= webdriver.Chrome(options=ch_options)
    with open('/Users/zuimingze/python练练/tools/stealth.min.js') as f:
        js = f.read()

    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": js
    })
    driver.delete_all_cookies()
    driver.implicitly_wait(60)
    driver.get(url)
    return driver

In [27]:
#i选择颜色，j选择容量
def chose_model(i:int=4,j:int=2):
    model_list={
     #14 pro
    #/html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div[1]/input
    #14 pro max
    #/html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div[2]/input
    "model":"/html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div[2]/input",   
    #紫色'/html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[2]/div/ul/li[1]/label/img
    #金色/html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[2]/div/ul/li[2]/label/img
    #银色/html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[2]/div/ul/li[3]/label/img
    #黑色/html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[2]/div/ul/li[4]/label/img
    "color":"/html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[2]/div/ul/li[{}]/label/img".format(i),   
          #手机容量
    #128g# /html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[1]
    #256g# /html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[2]
    #512g# /html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[3]
    #1t# /html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[4]
    "rom":"/html/body/div[2]/div[4]/div[4]/div[3]/div[2]/div[2]/div/div[3]/div[1]/div[{}]".format(j), 
                #不回收
    "no_trade":"/html/body/div[2]/div[4]/div[4]/div[3]/div[3]/div[1]/div[2]/div[1]/div/div/div/div[3]",
          #选择无apple care
    "no_care":"/html/body/div[2]/div[4]/div[4]/div[3]/div[3]/div[3]/div/div/div[1]/div[1]/fieldset/div/div/div[2]/label"
    #"add_bag":"/html/body/div[2]/div[4]/div[4]/div[3]/div[3]/div[4]/div/div/div/div/div/div[3]/div/div/div/div[2]/div/form/div/span/button"
    
    }
    for k in  model_list.keys():
        
        ele=browser.find_elements("xpath",model_list[k])
     
        ele[0].click()
        
    browser.refresh()
    add_bag='/html/body/div[2]/div[4]/div[4]/div[3]/div[3]/div[4]/div/div/div/div/div/div[3]/div/div/div/div[2]/div/div/form/div/span/button/span'
    #"/html/body/div[2]/div[4]/div[4]/div[3]/div[3]/div[4]/div/div/div/div/div/div[3]/div/div/div/div[2]/div/form/div/span/button"
    
   
    ele_add_bag=browser.find_elements("xpath",add_bag)
    ele_add_bag[0].click()
        

In [28]:
#中间网页
def middle_page():
    
    middle_page={
        "review_bag":"/html/body/div[2]/div[4]/div[2]/div/div/div/div[2]/div/form/button",
        "check_out":"/html/body/div[2]/div[4]/div/div[2]/div[1]/div[2]/div/div/div[1]/button",
        "buy_guest":"/html/body/div[2]/div[4]/div[1]/div/div[3]/div/div/button",
        "continue_ship_address":"/html/body/div[2]/div[4]/div/div[2]/div/div[1]/div[3]/div/div/button"
    }
    for k in middle_page.keys():
       
        ele=browser.find_elements("xpath",middle_page[k])
        ele[0].click()
        


In [29]:
#填写邮寄信息并截图保存
def ship_info():
    shipp_info_url_list={
        'first_name':'/html/body/div[2]/div[4]/div/div[2]/div[1]/div[2]/fieldset/div/div/div[1]/div/div/div[1]/div/div/div/div/div/div/input',
        'last_name':'/html/body/div[2]/div[4]/div/div[2]/div[1]/div[2]/fieldset/div/div/div[1]/div/div/div[2]/div/div/div/div/div/div/input',
        'st_addres':'/html/body/div[2]/div[4]/div/div[2]/div[1]/div[2]/fieldset/div/div/div[1]/div/div/div[3]/div/div/div/div/div/div/input',
    'add_addres':'/html/body/div[2]/div[4]/div/div[2]/div[1]/div[2]/fieldset/div/div/div[1]/div/div/div[4]/div/div/div/div/div/div/input',
    'email':'/html/body/div[2]/div[4]/div/div[2]/div[1]/fieldset/div[1]/div/div/div/div[1]/div/div/div/div/div/input',
    'tel':'/html/body/div[2]/div[4]/div/div[2]/div[1]/fieldset/div[2]/div/div/div[1]/div[1]/div/div/div/div/div/input',
    }
 
    for k in shipp_info_url_list.keys():
       
        ele=browser.find_elements("xpath",shipp_info_url_list[k])
        #time.sleep(random.uniform(3,5))
        ele[0].click()
        
        ele[0].send_keys(order_info[k])
        browser.execute_script("window.scrollBy(0,{})".format(random.uniform(10,20))) 
    #缩小图片截图报存,以下单邮箱号手机号
    browser.save_screenshot("{}_{}.png".format(order_info['email'],order_info['tel']))
    #前往结算页面
    go_pay='/html/body/div[2]/div[4]/div/div[2]/div[1]/div[4]/div/div/div/button'
    ele_go_pay=browser.find_elements("xpath",go_pay)
    browser.execute_script("window.scrollBy(0,{})".format(random.uniform(30,50)))
    
    ele_go_pay[0].click()

In [30]:
def add_pay():
    enter_gift='/html/body/div[2]/div[4]/div/div[2]/div/div[1]/div[2]/fieldset/div/div/div[2]/div/div/div/div/div/div[2]/div/div/button'
    ele_enter_gift=browser.find_elements("xpath",enter_gift)
    ele_enter_gift[0].click()
    gc_input='/html/body/div[2]/div[4]/div/div[2]/div/div[1]/div[2]/fieldset/div/div/div[2]/div/div/div/div/div/div[2]/div/div/div/div/div/div/div[1]/div/input'
    ele_gc_input=browser.find_elements("xpath",gc_input)
    ele_gc_input[0].send_keys(order_info['card'])
    gc_apply='/html/body/div[2]/div[4]/div/div[2]/div/div[1]/div[2]/fieldset/div/div/div[2]/div/div/div/div/div/div[2]/div/div/div/div/div/div/div[1]/div/button/span'
    ele_gc_apply=browser.find_elements("xpath",gc_apply)
    ele_gc_apply[0].click()
        


In [31]:
def re_place_order():
    #复核订单信息
    review_order='/html/body/div[2]/div[4]/div/div[2]/div/div[1]/div[3]/div/div'
    ele_review_order=browser.find_elements("xpath",review_order)
    
    ele_review_order[0].click()
   
    #提交订单
    browser.execute_script("window.scrollBy(0,{})".format(random.uniform(100,200)))
    time.sleep(random.uniform(1,3))
    place_order_url='/html/body/div[2]/div[4]/div/div[2]/div/div[1]/div[5]/div/div/div/button'
   
    ele_place_order=browser.find_elements("xpath",place_order_url)
    ele_place_order[0].click()
   
    ordernum_label='/html/body/div[3]/div[4]/div/div[1]/div/div/a'
    browser.find_elements("xpath",ordernum_label)
    #保险起见截图保存
    time.sleep(random.uniform(3,5))
    browser.save_screenshot('{}.png'.format(order_info['email']))
    order_num="/html/body/div[3]/div[4]/div/div[1]/div/div/a"
    ele_order_num=browser.find_elements("xpath",order_num)[0]
    return ele_order_num

In [32]:
#获取第i单下单资料，从0开始
def get_order_info(i:int=0):
    usecols=['email', 'tel','fstname','latname','st_address','ad_address','card']
    df1=pd.read_csv('/Users/zuimingze/python练练/order1222/order1222_info.csv',usecols=usecols)
    order_info=df1.loc[i]
    item_detail= { 
        "email":order_info[0],
         "tel":int(order_info[1]),
        "first_name":order_info[2],
          "last_name":order_info[3],
        "st_addres":order_info[4],
         "add_addres":order_info[5],
          "card":order_info[6]
    }
    return item_detail

In [118]:
i=59
order_info=get_order_info(i)
#第一步打开浏览器选择型号
browser=get_browser()
browser.refresh()
time.sleep(random.uniform(1,3))
#选择型号，默认紫色256
chose_model(1,2)
time.sleep(random.uniform(1,3))
#查看购物车并以访客模式购买
middle_page()
time.sleep(random.uniform(1,3))
#获取邮寄信息
ship_info()
time.sleep(random.uniform(1,3))
add_pay()
#复核订单并提交饭回订单号
time.sleep(random.uniform(1,3))
order_number=re_place_order()
#息保存
Note=open('order1222.txt',mode='a')
Note.writelines([order_info['first_name'],',',order_info['last_name'],',',order_info['st_addres'],',',
                 order_info['add_addres'],',',order_info['email'],',',str(order_info['tel']),',',
                 str(order_info['card']),',',order_number.text,'\n']) #\n 换行符
Note.close()
browser.delete_all_cookies()
browser.close()
time.sleep(10)

In [57]:
browser.delete_all_cookies()
browser.close()

In [40]:
get_browser('https://ipinfo.ipidea.io/')

<selenium.webdriver.chrome.webdriver.WebDriver (session="5ae92eb9ced72bdc9f621dee081237c3")>

In [90]:
browser.close()